# 패키지 다운로드

In [2]:
import pandas as pd 
import numpy as np 
import warnings

warnings.filterwarnings('ignore') # warning 문구 안보이게

# 데이터 로딩 및 전처리

## 독립변수

### HSI

In [113]:
# 데이터 읽어오기 : pd.read_csv('filepath')
HSI=pd.read_csv('./구리 수요/(수요) Housing_Starts_Index_May_2023.csv')

# 데이터 테이블 구조 확인 -> Unnamed:2 라는 불필요한 컬럼 존재 
HSI.head()
#HSI.columns

# 불필요한 컬럼 삭제: DataFrame.drop('축 이름',axis=0 or 1) 
HSI=HSI.drop('Unnamed: 2',axis=1)
HSI.rename(columns={'Value':'HSI_value'},inplace=True)
# 재확인 
HSI.head()

,Date,HSI_value
0,1990-01-31,1551
1,1990-02-28,1437
2,1990-03-31,1289
3,1990-04-30,1248
4,1990-05-31,1212


### CCI

In [112]:
CCI=pd.read_csv('./구리 수요/(수요) Consumer confidence index (CCI).csv')
CCI.head() 

CCI['TIME']=pd.to_datetime(CCI['TIME']) 

CCI.rename(columns={'TIME':'Date'}, inplace=True) 
CCI.rename(columns={'Value':'CCI_value'},inplace=True)
CCI

,Date,CCI_value
0,1990-01-01,102.05330
1,1990-02-01,101.93030
2,1990-03-01,101.84770
3,1990-04-01,101.73750
4,1990-05-01,101.60550
...,...,...
396,2023-01-01,97.72659
397,2023-02-01,98.16353
398,2023-03-01,98.40943
399,2023-04-01,98.43763


### IPI 

In [109]:
IPI=pd.read_csv('./구리 수요/(수요) Industrial Production Total Index_INDPRO.csv')
IPI.head()

# 컬럼명 변경 : 
IPI.rename(columns={'DATE':'Date'},inplace=True)
IPI.rename(columns={'INDPRO' : 'IPI_value'}, inplace=True) 
IPI.head()

,Date,IPI_value
0,1990-01-01,61.6440
1,1990-02-01,62.2108
2,1990-03-01,62.5068
3,1990-04-01,62.3597
4,1990-05-01,62.5474


### GDP

In [60]:
# TimeStamps 를 맞추기 위해 1월 -> 1월 / 2월 / 3월  단순 재가공 
temp=pd.read_csv('./구리 수요/(수요) 분기별 Real Gross Domestic Product (GDPC1)1990-2023_JAN.csv')

temp.rename(columns={'GDPC1':'GDPC_value'},inplace=True) # inplace=True 잊지말기
#print(temp.head()) 

temp["DATE"]=pd.to_datetime(temp['DATE'])


# 각 분기의 데이터를 월별로 확장한 후 합치기 
monthly_data=[]
for index, row in temp.iterrows():
    year=row['DATE'].year
    month=row['DATE'].month
    gdp=row['GDPC_value'] # 단순 재가공  
    for i in range(3):
        monthly_data.append({'Date':pd.Timestamp(year,month+i,1), 'GDPC_value':gdp})

# 재가공된 월별 데이터 프레임 생성
monthly_data=pd.DataFrame(monthly_data)
print(monthly_data.head())

        Date  GDPC_value
0 1990-01-01    9364.259
1 1990-02-01    9364.259
2 1990-03-01    9364.259
3 1990-04-01    9398.243
4 1990-05-01    9398.243


In [110]:
GDP=pd.read_csv('./구리 수요/(수요) 분기별 Real Gross Domestic Product (GDPC1)1990-2023_JAN.csv')

GDP.rename(columns={'GDPC1':'GDPC_value'},inplace=True)

GDP["DATE"]=pd.to_datetime(temp['DATE']) 

# 각 분기의 데이터를 월별로 확장한 후 합치기 
monthly_data=[]
for index, row in GDP.iterrows():
    year=row['DATE'].year
    month=row['DATE'].month
    gdp=row['GDPC_value'] # 단순 재가공  
    for i in range(3):
        monthly_data.append({'Date':pd.Timestamp(year,month+i,1), 'GDPC_value':gdp}) 

GDP=pd.DataFrame(monthly_data)
print(GDP.head())

        Date  GDPC_value
0 1990-01-01    9364.259
1 1990-02-01    9364.259
2 1990-03-01    9364.259
3 1990-04-01    9398.243
4 1990-05-01    9398.243


### 구리가격

In [127]:
CP=pd.read_csv('./구리 수요/(수요) copper-prices-historical-chart-data.csv')

# str -> datetime 형으로 바꾸기 
CP['date']=pd.to_datetime(CP['date']) 

print(CP.head())


# 일별 데이터를 월별 데이터로 바꿔야함.

# (기존 방식) 1990-01월부터 2023-03-31 까지의 data 만 살리고 나머지는 버림. 
CP=CP[CP['date']>='1990-01-01'][CP['date']<='2023-03-31'].reset_index(drop=True)

# (새로운 방식)
CP['month']=CP['date'].dt.to_period('M') # .dt? 


CP=CP.groupby('month').mean('value').reset_index(drop=True)
print(CP.head())  

CP.columns=['Copper price']
CP

        date   value
0 1990-01-02  1.0620
1 1990-01-03  1.0965
2 1990-01-04  1.0855
3 1990-01-05  1.0950
4 1990-01-08  1.1135
      value
0  1.380976
1  1.322179
2  1.351283
3  1.328911
4  1.260091


,Copper price
0,1.380976
1,1.322179
2,1.351283
3,1.328911
4,1.260091
...,...
334,3.656582
335,3.817891
336,4.118691
337,4.082400


## 종속변수

In [100]:
IND=pd.read_csv('./구리 수요/230622 Global Copper Demand.csv')
IND.head() 

,Dates,PX_LAST
0,1995-01-31,969917.0
1,1995-02-28,960039.0
2,1995-03-31,1043928.0
3,1995-04-30,1022904.0
4,1995-05-31,979813.0


# 데이터 병합
- 문제 : 종속변수의 data 가 1995년부터라 위에서 1990년 부터 갖고 온 데이터를 버려야 했음 -> 12*5 60 COLUMN 이후 부터 갖고옴 

In [130]:
# 독립변수 합치기
DEP = pd.concat([HSI[["HSI_value"]], CCI[["CCI_value"]], IPI[["IPI_value"]], GDP[["GDPC_value"]], CP[['Copper price']]], axis = 1)[60:].reset_index(drop=True)
 

# 독립변수 + 종속변수 테이블 생성하기 
total=pd.concat([DEP,IND],axis=1)
total=total[:-2]
total

,HSI_value,CCI_value,IPI_value,GDPC_value,Copper price,Dates,PX_LAST
0,1407,100.83360,71.2762,10550.251,0.855289,1995-01-31,969917.000
1,1316,100.94720,71.1904,10550.251,0.827855,1995-02-28,960039.000
2,1249,101.02100,71.2891,10550.251,0.805130,1995-03-31,1043928.000
3,1267,100.98750,71.1623,10581.723,0.776305,1995-04-30,1022904.000
4,1314,100.98590,71.5044,10581.723,0.831318,1995-05-31,979813.000
...,...,...,...,...,...,...,...
334,1427,96.77289,103.0707,20182.491,NaN,2022-11-30,2237807.240
335,1357,97.26849,101.4848,20182.491,NaN,2022-12-31,2148356.118
336,1340,97.72659,102.5080,20246.439,NaN,2023-01-31,2121282.622
337,1436,98.16353,102.5023,20246.439,NaN,2023-02-28,1975076.391


# 데이터 저장

In [131]:
total.to_csv('./구리 수요/1995_01-2023_03 수요 데이터 전처리')